In [1]:
# pip install pdfplumber nltk numpy rapidfuzz

In [3]:
import pdfplumber
import glob
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import regex as re
import base64
import numpy as np
from multiprocessing import Pool, cpu_count
import pickle
import os
from rapidfuzz import fuzz
from rapidfuzz import process
import json
from custom_bm25 import BM25Okapi


In [3]:

keywords_to_remove = (
    'Score:', 'Episodes', 'Status', 'Aired', 'Premiered', 'Broadcast', 'Licensors',
    'Source', 'Duration', 'Characters', 'Anime Details', 'Japanese', 'Main Supporting',
    'Main Main', 'None', 'Demographic', 'Genres', 'Studios',
    'Data source: MyAnimeList', 'Supporting Supporting', '[Written by MAL Rewrite]'
)

animes_corpus = []
animes_json = []

pdfs = glob.glob(r'C:\Users\menno\Desktop\AI-location\animes_search_engine\pdf_transformer\pdfs\*.pdf')

for index, pdf in enumerate(pdfs):
    with pdfplumber.open(pdf) as f:
        
        txt = ''.join(page.extract_text() for page in f.pages)
        list_sentences = txt.split('\n')
        
        # Filter sentences based on keywords
        filtered_sentences = [
            i for i in list_sentences if not any(i.startswith(keyword) for keyword in keywords_to_remove)
        ]
        animes_corpus.append('\n'.join(filtered_sentences))

        # Extract specific fields
        def find_value(key, sentences):
            for i, sentence in enumerate(sentences):
                if sentence.lower().startswith(key.lower()):
                    return i
            return None

        score_index = find_value('score:', list_sentences)
        genres_index = find_value('Genres', list_sentences)
        description_end_index = find_value('Anime Details',list_sentences)
        demographic_index = find_value('Demographic', list_sentences)
        studios_index = find_value('Studios', list_sentences)
        premiered_index = find_value('Premiered', list_sentences)

        # Handling fields
        def extract_field_value(index, label):
            if index is not None:
                parts = list_sentences[index].split(label)
                return parts[1].strip() if len(parts) > 1 else list_sentences[index + 1].strip()
            return None

        demographic_value = extract_field_value(demographic_index, 'Demographic')
        studios_value = extract_field_value(studios_index, 'Studios')
        premiered_value = extract_field_value(premiered_index, 'Premiered')

        # JSON format output per doc
        animes_json.append({
            'doc_name': os.path.basename(pdf),
            'title': f.metadata.get('Title'),
            # 'image':base64.b64encode(f.images[0]["stream"].get_data()).decode('utf-8')  ,
            'score': list_sentences[score_index].split()[1],
            'description': '\n'.join(list_sentences[score_index + 1:description_end_index]) if score_index is not None and description_end_index is not None else None,
            'genres': list_sentences[genres_index].split()[1:] if genres_index else None,
            'demographic': demographic_value,
            'studios': studios_value,
            'premiered': premiered_value
        })
        
    print(f'anime {index + 1}')


anime 1
anime 2
anime 3
anime 4
anime 5
anime 6
anime 7
anime 8
anime 9
anime 10
anime 11
anime 12
anime 13
anime 14
anime 15
anime 16
anime 17
anime 18
anime 19
anime 20
anime 21
anime 22
anime 23
anime 24
anime 25
anime 26
anime 27
anime 28
anime 29
anime 30
anime 31
anime 32
anime 33
anime 34
anime 35
anime 36
anime 37
anime 38
anime 39
anime 40
anime 41
anime 42
anime 43
anime 44
anime 45
anime 46
anime 47
anime 48
anime 49
anime 50
anime 51
anime 52
anime 53
anime 54
anime 55
anime 56
anime 57
anime 58
anime 59
anime 60
anime 61
anime 62
anime 63
anime 64
anime 65
anime 66
anime 67
anime 68
anime 69
anime 70
anime 71
anime 72
anime 73
anime 74
anime 75
anime 76
anime 77
anime 78
anime 79
anime 80
anime 81
anime 82
anime 83
anime 84
anime 85
anime 86
anime 87
anime 88
anime 89
anime 90
anime 91
anime 92
anime 93
anime 94
anime 95
anime 96
anime 97
anime 98
anime 99
anime 100
anime 101
anime 102
anime 103
anime 104
anime 105
anime 106
anime 107
anime 108
anime 109
anime 110
anime 11

In [4]:
# print(animes_corpus)

In [5]:

# Save animes_json to a file
with open('animes_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(animes_json, json_file, ensure_ascii=False, indent=4)

print("animes_json has been saved to 'animes_data.json'")


animes_json has been saved to 'animes_data.json'


In [4]:
nltk.download('stopwords')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\menno\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\menno\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
#lower, remove punctuations , remove stop words , tokenize input
def filter_text(input_string , string , stop_words):
    tokenized_input = input_string.translate(str.maketrans('', '', string.punctuation)).lower().split()
    filterd_tokenized_input = [w for w in tokenized_input if not w in stop_words]
    filterd_tokenized_input = [ps.stem(lemmatizer.lemmatize(i)) for i in filterd_tokenized_input ]
    return filterd_tokenized_input


In [8]:
filter_text(animes_corpus[0] , string , stop_words)

['gintama',
 'gintama',
 'season',
 '5',
 'join',
 'resist',
 'bakufu',
 'gintoki',
 'gang',
 'hide',
 'along',
 'katsura',
 'joui',
 'rebel',
 'yorozuya',
 'soon',
 'approach',
 'nobum',
 'imai',
 'two',
 'member',
 'kiheitai',
 'explain',
 'harusam',
 'pirat',
 'turn',
 '7th',
 'divis',
 'captain',
 'kamui',
 'former',
 'alli',
 'takasugi',
 'kiheitai',
 'present',
 'gintoki',
 'job',
 'find',
 'takasugi',
 'miss',
 'sinc',
 'ship',
 'ambush',
 'harusam',
 'raid',
 'nobum',
 'also',
 'make',
 'stun',
 'revel',
 'regard',
 'tendoushuu',
 'secret',
 'organ',
 'pull',
 'string',
 'numer',
 'faction',
 'leader',
 'utsuro',
 'shadowi',
 'figur',
 'uncanni',
 'resembl',
 'gintoki',
 'former',
 'teacher',
 'hitch',
 'ride',
 'sakamoto',
 'space',
 'ship',
 'yorozuya',
 'katsura',
 'set',
 'rakuy',
 'kagura',
 'home',
 'planet',
 'variou',
 'faction',
 'gather',
 'tension',
 'brew',
 'longheld',
 'grudg',
 'polit',
 'infight',
 'tendoushuu',
 'sinist',
 'overarch',
 'plan',
 'final',
 'culmi

In [9]:
tokenized_cleaned_corpus = []
for doc in animes_corpus:
    clean_doc = filter_text(doc , string , stop_words)
    tokenized_cleaned_corpus.append(clean_doc)


In [10]:
tokenized_cleaned_corpus

[['gintama',
  'gintama',
  'season',
  '5',
  'join',
  'resist',
  'bakufu',
  'gintoki',
  'gang',
  'hide',
  'along',
  'katsura',
  'joui',
  'rebel',
  'yorozuya',
  'soon',
  'approach',
  'nobum',
  'imai',
  'two',
  'member',
  'kiheitai',
  'explain',
  'harusam',
  'pirat',
  'turn',
  '7th',
  'divis',
  'captain',
  'kamui',
  'former',
  'alli',
  'takasugi',
  'kiheitai',
  'present',
  'gintoki',
  'job',
  'find',
  'takasugi',
  'miss',
  'sinc',
  'ship',
  'ambush',
  'harusam',
  'raid',
  'nobum',
  'also',
  'make',
  'stun',
  'revel',
  'regard',
  'tendoushuu',
  'secret',
  'organ',
  'pull',
  'string',
  'numer',
  'faction',
  'leader',
  'utsuro',
  'shadowi',
  'figur',
  'uncanni',
  'resembl',
  'gintoki',
  'former',
  'teacher',
  'hitch',
  'ride',
  'sakamoto',
  'space',
  'ship',
  'yorozuya',
  'katsura',
  'set',
  'rakuy',
  'kagura',
  'home',
  'planet',
  'variou',
  'faction',
  'gather',
  'tension',
  'brew',
  'longheld',
  'grudg',
 

In [20]:
def flatten_and_remove_duplicates(nested_list):
    # Flatten the list using recursion
    def flatten(lst):
        for item in lst:
            if isinstance(item, list):
                yield from flatten(item)
            else:
                yield item

    # Flatten the list and convert it to a set to remove duplicates
    flattened = list(flatten(nested_list))
    unique_items = []
    for item in flattened:
        if len(item) > 2 and item not in unique_items:  # Skip items with length 1 and 2
            unique_items.append(item)

    return unique_items


# flatten_corpus = flatten_and_remove_duplicates(tokenized_cleaned_corpus)

list_to_flatten = []

for doc in animes_corpus:
    clean_doc = doc.translate(str.maketrans('', '', string.punctuation)).lower().split()
    list_to_flatten.append(clean_doc)


flatten_corpus = flatten_and_remove_duplicates(list_to_flatten)

In [21]:
with open('flatten_corpus.json', 'w', encoding='utf-8') as json_file:
    json.dump(flatten_corpus, json_file, ensure_ascii=False)

In [22]:
bm25 = BM25Okapi(tokenized_cleaned_corpus)

In [24]:
# # # save model
file_name = r"models\model.pkl"

os.makedirs(os.path.dirname(file_name),exist_ok=True)

with open(file_name, 'wb') as file:
    pickle.dump(bm25, file)

In [6]:
#loading the bm25 model
with open(r"models\model.pkl", 'rb') as file:
    model = pickle.load(file)

# loading the flatten corpus for the fuzzy search
with open('flatten_corpus.json', 'r', encoding='utf-8') as json_file:
    flatten_corpus = json.load(json_file)

# loading the the pdfs jsons : results
with open('animes_data.json', 'r', encoding='utf-8') as json_file:
    animes_json = json.load(json_file)

doc_names = [anime['doc_name'] for anime in animes_json]




In [ ]:



# query = "one one one one one"
# tokenized_cleaned_query = filter_text(query , string , stop_words)

# fuzzy_tokenized_cleaned_query = []

# for q in tokenized_cleaned_query:
#     if len(q) <= 2:
#         q=q
#     else :
#         q = process.extractOne(q, flatten_corpus)[0]
#         print(process.extract(q,flatten_corpus,limit=2))
#     fuzzy_tokenized_cleaned_query.append(q)

# print(fuzzy_tokenized_cleaned_query)


# result = model.get_top_n(fuzzy_tokenized_cleaned_query , doc_names, n = len(doc_names))
# print(result)


['human', 'car', 'not', 'universe']


In [9]:

query = "the straw hats pirates "

tokenized_query = query.translate(str.maketrans('', '', string.punctuation)).lower().split()

fuzzy_tokenized_query_list = []

for q in tokenized_query:
    if len(q) <= 2:
        q=q
    else :
        fuzzy_query = process.extractOne(q, flatten_corpus)
        q = fuzzy_query[0] if fuzzy_query[1] > 79 else q
        
    fuzzy_tokenized_query_list.append(q)

fuzzy_tokenized_query = ' '.join(fuzzy_tokenized_query_list)
print(fuzzy_tokenized_query)

fuzzy_tokenized_cleaned_query = filter_text(fuzzy_tokenized_query , string , stop_words)
print(fuzzy_tokenized_cleaned_query)

result = model.get_top_n(fuzzy_tokenized_cleaned_query , doc_names, n = 100)
print(result)


the straw hats pirates
['straw', 'hat', 'pirat']
[{'doc': 'anime125.pdf', 'score': 22.60586306937485}, {'doc': 'anime394.pdf', 'score': 22.447793597859977}, {'doc': 'anime194.pdf', 'score': 8.667714384464038}, {'doc': 'anime378.pdf', 'score': 7.792221817980106}, {'doc': 'anime393.pdf', 'score': 6.622733925575379}, {'doc': 'anime199.pdf', 'score': 4.283038580021359}, {'doc': 'anime1.pdf', 'score': 3.968655687843786}]
